<center>
<img src="../../img/ods_stickers.jpg">
## Открытый курс по машинному обучению. Сессия № 3
</center>
Авторы материала: аспирант Мехмата МГУ Евгений Колмаков, программист-исследователь Mail.ru Group Юрий Кашницкий. Материал распространяется на условиях лицензии [Creative Commons CC BY-NC-SA 4.0](https://creativecommons.org/licenses/by-nc-sa/4.0/). Можно использовать в любых целях (редактировать, поправлять и брать за основу), кроме коммерческих, но с обязательным упоминанием автора материала.

# <center>Домашнее задание № 3. Опциональная часть 
## <center> Реализация алгоритма построения дерева решений

In [4]:
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
from sklearn.base import BaseEstimator
from sklearn.datasets import make_classification, make_regression, load_digits, load_boston
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, mean_squared_error

**Необходимо реализовать класс `DecisionTree`**

**Спецификация:**
- класс наследуется от `sklearn.BaseEstimator`;
- конструктор содержит следующие параметры: 
    `max_depth` - максимальная глубина дерева (по умолчанию - `numpy.inf`); 
    `min_samples_split` - минимальное число объектов в вершине, при котором происходит её разбиение (по умолчанию - 2); 
    `criterion` - критерий разбиения (для классификации - 'gini' или 'entropy', для регрессии - 'variance' или 'mad_median'; 
    по умолчанию - 'gini');
    
    Функционал, значение которого максимизируется для поиска оптимального разбиения в данной вершине имеет вид
    $$Q(X, j, t) = F(X) - \dfrac{|X_l|}{|X|} F(X_l) - \dfrac{|X_r|}{|X|} F(X_r),$$
    где $X$ - выборка, находящаяся в текущей вершине, $X_l$ и $X_r$ - разбиение выборки $X$ на две части 
    по предикату $[x_j < t]$, а $F(X)$ -критерий разбиения.
    
    Для классификации: пусть $p_i$ - доля объектов $i$-го класса в выборке $X$.
    
    'gini': Неопределенность Джини $F(X) = 1 -\sum_{i = 1}^K p_i^2$.
    
    'entropy': Энтропия $F(X) = -\sum_{i = 1}^K p_i \log_2(p_i)$.
    
    Для регрессии: $y_j = y(x_j)$ - ответ на объекте $x_j$, $y = (y_1, \dots, y_{|X|})$ - вектор ответов.
    
    'variance': Дисперсия (среднее квадратичное отклонение от среднего) $F(X) = \dfrac{1}{|X|} \sum_{x_j \in X}(y_j - \dfrac{1}{|X|}\sum_{x_i \in X}y_i)^2$
    
    'mad_median': Среднее отклонение от медианы $F(X) = \dfrac{1}{|X|} \sum_{x_j \in X}|y_j - \mathrm{med}(y)|$
    
- класс имеет методы `fit`, `predict` и `predict_proba`;

- ---
- метод `fit` принимает матрицу объектов `X` и вектор ответов `y` (объекты `numpy.ndarray`) и возвращает экземпляр класса
    `DecisionTree`, представляющий собой решающее дерево, обученное по выборке `(X, y)` с учётом заданных в конструкторе параметров; 
------
- метод `predict_proba` принимает матрицу объектов `X` и возвращает матрицу `P` размера `X.shape[0] x K`, где `K` - число классов, такую что $p_{ij}$ есть вероятность принадлежности объекта, заданного $i$-ой строкой матрицы X к классу $j \in \{1, \dots, K\}$.
_______
- метод `predict` принимает матрицу объектов и возвращает вектор предсказанных ответов; в случае классификации - это 
    наиболее многочисленный класс в листе, в который попал объект, а в случае регрессии - среднее значение ответов по 
    всем объектам этого листа;

In [125]:
def entropy(y):    
    pass

def gini(y):
    pass

def variance(y):
    '''
    $F(X) = \dfrac{1}{|X|} \sum_{x_j \in X}(y_j - \dfrac{1}{|X|}\sum_{x_i \in X}y_i)^2$
    '''
    l = len(y)
    if l == 0:
        return 0
    mean = (1/l)*np.sum(y)
    s = 0
    for i in range(l):
        s += (y[i]-mean)**2
    return (1/l)*s

def mad_median(y):
    l = len(y)
    med = np.median(y)
    s = 0
    for i in range(l):
        s += np.abs(y[i]-med)
    return (1/l)*s


In [153]:
#test

X = np.linspace(-2, 2, 50)
y = X ** 3


In [165]:
class DecisionTree(BaseEstimator):

    class Node:
        
        def __init__(self, value=None, left=None, right=None):
            self.left = left
            self.right = right
            self.value = value
    
        def set_val(self, val):
            self.value = val
    
        def set_left(self, val_l):
            self.left = val_l
            
        def set_right(self, val_r):
            self.right = val_r
            
        def get_left(self):
            return self.left
    
        def get_right(self):
            return self.right
        
        def has_left(self):
            return self.left != None
        
        def has_right(self):
            return self.right != None
    
    def __init__(self, max_depth=np.inf, min_samples_split=2, 
                 criterion='variance', debug=False):
        self.criterias = { 
                    'variance':variance,
                    'mad_median':mad_median
                }
        self.min_samples_split = min_samples_split
        self.crit_ = self.criterias[criterion]
        self.root_node = DecisionTree.Node()
    
    
    def find_better_split(self, X, y):
        if len(X) == 0:
            return 0, 0, 0
        l = X.shape[0]
        max_crit= -np.inf
        t = X[0]
        position = 0
        for i in range(0,l):
            Q = self.criterion_(X,y,X[i])
            #print(Q)
            if Q >= max_crit:
                max_crit = Q
                t = X[i]
                position = i
        return max_crit, t, position
    
    def split(self, X, y, position, rootNode):
        if len(X) < self.min_samples_split:
            return
        root_X = X[position]
        #set vale
        rootNode.set_val(root_X)
        print('ROOT_X:', root_X)
        X_l = X[:position-1]
        y_l = y[:position-1]
        
        X_r = X[position+1:]
        y_r = y[position+1:]
        
        #go left
        Q_l, t_l, position_l = self.find_better_split(X_l, y_l)
        left_node = DecisionTree.Node()
        rootNode.set_left(left_node)
        print('LEFT')
        self.split(X_l, y_l, position_l, left_node)
        print('LEFT_END')
        
        #go right
        Q_r, t_r, position_r = self.find_better_split(X_r, y_r)
        right_node = DecisionTree.Node()
        rootNode.set_right(right_node)
        print('RIGHT')
        self.split(X_r, y_r, position_r, right_node)
        print('RIGHT_END')    

    
    def fit(self, X, y):
        #Initial split
        Q, t, position = self.find_better_split(X, y)
        
        self.split(X, y, position, self.root_node)
        
        print(self.root_node)
        
        
    def predict(self, X):
        
        pass
        
    def predict_proba(self, X):
        pass
    
    def criterion_(self, X, y, t):
        FX = self.crit_
        y_l = y[X<t]
        y_r = y[~(X<t)]
        capacity_X = X.shape[0]
        capacity_y_l = y_l.shape[0]
        capacity_y_r = y_r.shape[0]
        return FX(y) - (capacity_y_l/capacity_X)*FX(y_l) - (capacity_y_r/capacity_X)*FX(y_r)

In [166]:
print('FULL',X)
tree = DecisionTree(criterion='variance')
tree.fit(X, y)

FULL [-2.         -1.91836735 -1.83673469 -1.75510204 -1.67346939 -1.59183673
 -1.51020408 -1.42857143 -1.34693878 -1.26530612 -1.18367347 -1.10204082
 -1.02040816 -0.93877551 -0.85714286 -0.7755102  -0.69387755 -0.6122449
 -0.53061224 -0.44897959 -0.36734694 -0.28571429 -0.20408163 -0.12244898
 -0.04081633  0.04081633  0.12244898  0.20408163  0.28571429  0.36734694
  0.44897959  0.53061224  0.6122449   0.69387755  0.7755102   0.85714286
  0.93877551  1.02040816  1.10204082  1.18367347  1.26530612  1.34693878
  1.42857143  1.51020408  1.59183673  1.67346939  1.75510204  1.83673469
  1.91836735  2.        ]
ROOT_X: 1.1836734693877546
LEFT
ROOT_X: -1.3469387755102042
LEFT
ROOT_X: -1.7551020408163265
LEFT
ROOT_X: -1.9183673469387754
LEFT
LEFT_END
RIGHT
RIGHT_END
LEFT_END
RIGHT
ROOT_X: -1.5918367346938775
LEFT
LEFT_END
RIGHT
RIGHT_END
RIGHT_END
LEFT_END
RIGHT
ROOT_X: -0.7755102040816328
LEFT
ROOT_X: -1.1020408163265307
LEFT
LEFT_END
RIGHT
ROOT_X: -0.9387755102040818
LEFT
LEFT_END
RIGHT
RIG

## Тестирование реализованного алгоритма

### Классификация

С помощью метода `load_digits` загрузите датасет `digits`. Разделите выборку на обучающую и тестовую с помощью метода `train_test_split`, используйте значения параметров `test_size=0.2`, `random_state=17`. Попробуйте обучить неглубокие решающие деревья и убедитесь, что критерии gini и entropy дают разные результаты.

In [ ]:
# Ваш код здесь

С помощью 5-кратной кросс-валидации (`GridSearchCV`) подберите оптимальное значение параметров `max_depth` и `criterion`. Для параметра `max_depth` используйте диапазон значений - range(3, 11), а для criterion - {'gini', 'entropy'}. Критерий качества `scoring`='accuracy'.

In [ ]:
# Ваш код здесь

Постройте графики средних значений критерия качества `accuracy` для критериев `gini` и `entropy` в зависимости от `max_depth`.

In [ ]:
# Ваш код здесь

**Выберите верные утверждения:**
1. Оптимальное значение max_depth для каждого критерия достигается внутри отрезка [3, 10] (то есть на отрезке [4, 9]).
2. На отрезке [3, 10] построенные графики не пересекаются.
3. На отрезке [3, 10] построенные графики пересекаются ровно один раз.
4. Наилучшее качество при max_depth на интервале [3, 10] достигается на критерии gini.
5. Хотя бы для одного из критериев значение accuracy строго возрастает с ростом значения max_depth на интервале [3, 10].

**Чему равны найденные оптимальные значения параметров max_depth и criterion?**
1. max_depth = 4, criterion = 'gini';
2. max_depth = 7, criterion = 'entropy';
3. max_depth = 10, criterion = 'entropy';
4. max_depth = 10, criterion = 'gini';
5. max_depth = 3, criterion = 'entropy';
6. max_depth = 9, criterion = 'gini';

Используя найденные оптимальные значения max_depth и criterion, обучите решающее дерево на X_train, y_train и вычислите вероятности принадлежности к классам для X_test.

In [ ]:
# Ваш код здесь

Для полученной матрицы вычислите усредненные по всем объектам из `X_test` значения вероятностей принадлежности к классам.

**Вопрос:** Чему равна максимальная вероятность в полученном векторе?
1. 0.11218791
2. 0.11783761
3. 1.0
4. 0.0875

### Регрессия

С помощью метода `load_boston` загрузите датасет `boston`. Разделите выборку на обучающую и тестовую с помощью метода `train_test_split`, используйте значения параметров `test_size=0.2`, `random_state=17`. Попробуйте обучить неглубокие регрессионные деревья и убедитесь, что критерии `variance` и `mad_median` дают разные результаты.

In [ ]:
# Ваш код здесь

С помощью 5-кратной кросс-валидации подберите оптимальное значение параметров `max_depth` и `criterion`. Для параметра `max_depth` используйте диапазон значений - `range(2, 9)`, а для `criterion` - {'variance', 'mad_median'}. Критерий качества `scoring`='neg_mean_squared_error'.

In [ ]:
# Ваш код здесь

Постройте графики средних значений критерия качества `neg_mean_squared_error` для критериев `variance` и `mad_median` в зависимости от `max_depth`.

In [ ]:
# Ваш код здесь

**Выберите верные утверждения:**
1. На отрезке [2, 8] построенные графики не пересекаются.
2. На отрезке [2, 8] построенные графики пересекаются ровно один раз.
3. Оптимальное значение max_depth для каждого из критериев достигается на границе отрезка [2, 8].
4. Наилучшее качество при max_depth in range(2, 9) достигается на критерии variance.
5. График качества ровно для одного из критериев имеет явно выраженный пик.

**Чему равны найденные оптимальные значения параметров `max_depth` и `criterion`?**
1. max_depth = 9, criterion = 'variance';
2. max_depth = 5, criterion = 'mad_median';
3. max_depth = 4, criterion = 'variance';
4. max_depth = 2, criterion = 'mad_median';
5. max_depth = 4, criterion = 'mad_median';
6. max_depth = 8, criterion = 'variance';